In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/data/notebook_files/report (1).csv")
df.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
0,137.218750,45.390655,0.034985,0.134409,3.204013,17.177857,7.494588,74.003104,0
1,93.773438,41.883062,0.371402,1.291232,1.482441,12.959050,12.242911,180.158844,0
2,73.632812,35.461218,1.153607,4.161082,2.099498,13.057474,10.013591,140.894151,0
3,93.429688,39.883166,0.602060,1.518548,2.475753,14.146578,8.770640,107.970173,0
4,131.625000,44.111896,0.108905,0.239879,33.888796,70.398342,1.689460,1.031847,0


## 1. The task is to draw the sample from the  table using four queries based on the following criteria:
- All rows of the table where `TARGET = 0` and `MIP in [82.9453125, 84.3515625]`
- All rows of the table where `TARGET = 1` and `MIP in [83.7734375, 92.6015625]`

In [3]:
sample_df = df.query(
    "TARGET == 0 and MIP >= 82.9453125 and MIP <= 84.3515625 or TARGET == 1 and MIP >= 83.7734375 and MIP <= 92.6015625"
)
sample_df.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
187,84.742188,40.007274,1.368957,3.812073,5.736622,32.047311,5.979603,36.147975,1
190,91.828125,34.111744,1.446682,5.265289,3.741639,25.317370,7.235101,53.582074,1
280,83.679688,36.379281,0.572532,2.664611,4.040970,23.169129,7.006681,53.514005,0
342,84.039062,38.943951,0.668007,1.586058,1.793478,13.494579,11.433532,161.907833,0
385,84.273438,38.399975,0.662579,1.778425,2.984114,16.690291,8.407608,86.913036,0


## 2. Perform a linear normalization of all predictor values in the obtained sample. In Azure ML Studio, specify the Transformation method — MinMax in the parameters of Normalize Data item.

> Find the sample mean for the `MIP` column after the normalization:

In [4]:
normalized_df = (sample_df - sample_df.min()) / (sample_df.max() - sample_df.min())
normalized_df.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
187,0.186084,0.191667,0.453250,0.356576,0.033361,0.247052,0.285097,0.061703,1.0
190,0.919903,0.053336,0.486852,0.465047,0.020696,0.175490,0.336877,0.090023,1.0
280,0.076052,0.106541,0.108946,0.270927,0.022596,0.152646,0.327457,0.089913,0.0
342,0.113269,0.166718,0.150221,0.190422,0.008328,0.049772,0.510031,0.265987,0.0
385,0.137540,0.153954,0.147875,0.204781,0.015887,0.083754,0.385235,0.144166,0.0


In [5]:
normalized_df.MIP.mean()

0.29451840815149477

In [6]:
from sklearn.linear_model import LogisticRegression

X = normalized_df.drop(columns=["TARGET"])
y = normalized_df["TARGET"]

clf = LogisticRegression(random_state=2019).fit(X, y)

## 4. Classify a new star with the parameters:
```
[0.337, 0.513, 0.425, 0.036, 0.928, 0.721, 0.714, 0.261]
```

Enter the the probability of assigning the star to the pulsar class:

In [7]:
list(X.columns.values)

['MIP', 'STDIP', 'EKIP', 'SIP', 'MC', 'STDC', 'EKC', 'SC']

In [8]:
star_X = [0.337, 0.513, 0.425, 0.036, 0.928, 0.721, 0.714, 0.261]
stars_df = pd.DataFrame(star_X).T
# stars_df.head()

# add columns
stars_df.columns = list(X.columns.values)
stars_df.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC
0,0.337,0.513,0.425,0.036,0.928,0.721,0.714,0.261


In [9]:
# Probability pulsar class
clf.predict_proba(stars_df)

array([[0.38648584, 0.61351416]])

## 5. Classify a new star using the -nearest neighbors algorithm (k-NN).

In [13]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Instantiate and fit NearestNeighbors
k = 1  # Number of nearest neighbors to consider
nn = NearestNeighbors(n_neighbors=k, metric='euclidean')
nn.fit(X)

# Find distance to nearest neighbor
distances, indices = nn.kneighbors(stars_df)
nearest_distance = distances[0][0]

print(f"Distance to nearest neighbor: {nearest_distance}")

Distance to nearest neighbor: 0.7828460858809096
